In [1]:
import DES
import numpy as np
import simpy as sip
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import statistics as st

In [2]:
name = 'servers_'
mu, lamd = 1, 99/100
# print(f"rho = {lamd/(1/mu)}")
DES.run_simu(name, sip.Resource, mu, lamd, 1000)
res_dict, cutoffs = DES.waiting_times('servers_normal_1000')

rho = 0.99
1_servers_normal_1000


KeyboardInterrupt: 

In [ ]:
def plots(res_dict):
    c = [_, 'blue', 'green', _, 'orange']
    for n in [1,2,4]:
        results = res_dict[n]
        ct = cutoffs[n]
        means_n = [np.mean(results[ct[i]:ct[i+1]]) for i in range(len(ct)-1)]
        means_n.append(np.mean(results[ct[-2]:ct[-1]]))

        # Normal distribution plots
        means_n[means_n == 0] = np.nan

        mu, std = np.nanmean(means_n), np.nanstd(means_n)
        print(mu, std)
        x = np.linspace(mu - 3*std, mu + 5*std, 75)
        fig, ax = plt.subplots()
        plt.plot(x, stats.norm.pdf(x, mu, std), label=f'{n} server normal distribution', color=c[n])
        print(f"{n} server mean waiting times: {mu}")
        

        # Chi 2 distribution plots
        # df = st.median(means_n)
        # chi_x = np.linspace(stats.chi2.ppf(0.00001, df), stats.chi2.ppf(0.99999, df), 100)
        # plt.plot(chi_x, stats.chi2.pdf(chi_x, df), label=f'{n} server chi2 distribution', color='black')

        # Histogram
        plt.hist(means_n, bins=int(len(means_n)/10), label=f'{n} server means', alpha=0.5, color=c[n], density=True,stacked=True)
        ax.set_xlim(0,150)
        plt.legend()
        plt.show()

In [ ]:
plots(res_dict)

In [ ]:
# Waiting time distributions
plt.hist(res_dict[1], bins=100, label='1 server')
plt.hist(res_dict[2], bins=100, alpha=0.5, label='2 servers')
plt.hist(res_dict[4], bins=100, alpha=0.4, label='4 servers')
plt.legend()
plt.show()